# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127059e+02     1.752278e+00
 * time: 1.0874030590057373
     1     1.125994e+01     9.819044e-01
 * time: 3.2610220909118652
     2    -1.222361e+01     1.188076e+00
 * time: 3.9043240547180176
     3    -3.403371e+01     8.024268e-01
 * time: 4.828690052032471
     4    -4.764209e+01     5.941642e-01
 * time: 5.752012014389038
     5    -5.700607e+01     2.370731e-01
 * time: 6.666948080062866
     6    -5.976195e+01     2.238247e-01
 * time: 7.292791128158569
     7    -6.089367e+01     1.155579e-01
 * time: 7.908681154251099
     8    -6.141385e+01     3.913664e-02
 * time: 8.53733205795288
     9    -6.166661e+01     3.500688e-02
 * time: 9.14994215965271
    10    -6.185658e+01     2.944394e-02
 * time: 9.761003017425537
    11    -6.198052e+01     2.113292e-02
 * time: 10.37689208984375
    12    -6.206236e+01     1.633363e-02
 * time: 10.986956119537354
    13    -6.210672e+01     1.468277e-02
 * time: 11.60850214958191
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671082
    AtomicLocal         -18.8557724
    AtomicNonlocal      14.8522665
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485394 
    Xc                  -19.3336829

    total               -62.261666460978
